<a href="https://colab.research.google.com/github/timbyles/QuantDashboard/blob/main/Get_Financial_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoofinancials


In [ ]:
import pandas as pd
import concurrent.futures as cf
import time
import ast
from yahoofinancials import YahooFinancials

In [ ]:
urlSymbols = 'https://raw.githubusercontent.com/timbyles/ASX-Cointegration/main/ASX300.csv'
stockDF = pd.read_csv(urlSymbols)

balanceSheet = {}
incomeStatement = {}
cashStatement = {}

In [ ]:
# No Multithreading

stockList = ['A2M.AX', 'WOW.AX'] # Testing 2 instruments

start = time.time()

for stock in stockList:
  print(stock)
  yahoo_financials = YahooFinancials(stock)
  balance_sheet_data = yahoo_financials.get_financial_stmts('annual', 'balance')
  income_statement_data = yahoo_financials.get_financial_stmts('annual', 'income')
  cash_statement_data = yahoo_financials.get_financial_stmts('annual', 'cash')

  balanceSheet[stock] = balance_sheet_data['balanceSheetHistory'][stock]
  incomeStatement[stock] = income_statement_data['incomeStatementHistory'][stock]
  cashStatement[stock] = cash_statement_data['cashflowStatementHistory'][stock]

print(balanceSheet)

end = time.time()

totalTime = end - start  # Time to pull API data for 2 insturments = 42 seconds. Projected time = 42*150 = 6300
print(totalTime)


In [ ]:
# Multithreading* GIL vs Multiprocessing

stockList = stockDF['Code'].values.tolist()
stockList = [stock + '.AX' for stock in stockList]

def retrieve_stock_data(stock):
    try:
        print(stock)
        yahoo_financials = YahooFinancials(stock)
        balance_sheet_data = yahoo_financials.get_financial_stmts('annual', 'balance')
        income_statement_data = yahoo_financials.get_financial_stmts('annual', 'income')
        cash_statement_data = yahoo_financials.get_financial_stmts('annual', 'cash')

        balanceSheet[stock] = balance_sheet_data['balanceSheetHistory'][stock]
        incomeStatement[stock] = income_statement_data['incomeStatementHistory'][stock]
        cashStatement[stock] = cash_statement_data['cashflowStatementHistory'][stock]

    except:
        print('Error')

start = time.time()
executor = cf.ThreadPoolExecutor(16)
futures = [executor.submit(retrieve_stock_data, stock) for stock in stockList]
cf.wait(futures)
end = time.time()

totalTime = end - start  # Time to pull API data for 300 insturments = Approx. 400 seconds
print(totalTime)

balanceSheet

In [ ]:
with open('balanceSheet_ASX300.txt', 'w') as output:
  output.write(str(balanceSheet))
with open('incomeStatement_ASX300.txt', 'w') as output:
  output.write(str(incomeStatement))
with open('cashStatement_ASX300.txt', 'w') as output:
  output.write(str(cashStatement))

In [ ]:
with open('balanceSheet_ASX300.txt', 'r') as input:
  balanceSheet = ast.literal_eval(input.read())

with open('cashStatement_ASX300.txt', 'r') as input:
  cashStatement = ast.literal_eval(input.read())

with open('incomeStatement_ASX300.txt', 'r') as input:
  incomeStatement = ast.literal_eval(input.read())